In [1]:
# ! pip install numpy pillow svgwrite

# Bitmap output handler

In [2]:
import csv
import numpy as np
from PIL import Image

In [3]:
def read_csv(file_path):
    bucket_indices = []
    values = []
    with open(file_path, mode='r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header
        for row in reader:
            bucket_indices.append(int(row[0]))
            values.append(float(row[1]))
    return bucket_indices, values

# indices, values = read_csv(BITMAP_FILENAME)

In [4]:
def normalize(values):
    max_value = max(values)
    min_value = min(values)
    if max_value == min_value:  # Avoid division by zero
        return [0 for _ in values]
    return [int(255 - 255 * (value - min_value) / (max_value - min_value)) for value in values]

In [7]:
import svgwrite
import math

def reshape_to_square(values):
    # Find the smallest possible grid dimensions (width and height)
    total_values = len(values)
    width = math.ceil(math.sqrt(total_values))
    height = math.ceil(total_values / width)

    # Pad values with zeros if necessary
    padded_values = values + [0] * (width * height - total_values)

    # Reshape into a 2D grid (height x width)
    grid = np.array(padded_values).reshape(height, width)
    return grid

def create_svg_square(grid, output_path, max_size=350, max_pixels=1_000_000):
    """
    Creates an SVG representation of a 2D grid with grayscale intensity,
    ensuring the final image fits within a 1040x1040 canvas.
    
    Parameters:
        grid (numpy.ndarray): A 2D numpy array of values.
        output_path (str): Path to save the SVG file.
        max_size (int): Maximum size (in pixels) for the SVG canvas.
        max_pixels (int): Maximum number of pixels to render (for performance).
    """
    # Get original grid dimensions
    height, width = grid.shape
    total_pixels = height * width

    # Downscale if too many pixels to handle
    if total_pixels > max_pixels:
        scale_factor = (total_pixels / max_pixels) ** 0.5
        new_height = int(height / scale_factor)
        new_width = int(width / scale_factor)
        grid = grid[::int(scale_factor), ::int(scale_factor)]  # Simple downscaling
        height, width = grid.shape

    # Determine the pixel size to fit within the max_size canvas
    pixel_size = min(max_size / width, max_size / height)

    # Calculate the overall canvas size
    canvas_width = width * pixel_size
    canvas_height = height * pixel_size

    # Create an SVG drawing with the computed size
    dwg = svgwrite.Drawing(output_path, size=(canvas_width, canvas_height))

    # Normalize grid values to 0-255 for grayscale
    grid_min, grid_max = grid.min(), grid.max()
    normalized_grid = ((grid - grid_min) / (grid_max - grid_min) * 255).astype(int)

    # Add rectangles for each cell in the grid
    for y in range(height):
        for x in range(width):
            intensity = normalized_grid[y, x]
            color = f"rgb({intensity},{intensity},{intensity})"  # Grayscale color
            dwg.add(dwg.rect(
                insert=(x * pixel_size, y * pixel_size),
                size=(pixel_size, pixel_size),
                fill=color
            ))

    # Save the SVG file
    dwg.save()
    print(f"SVG saved to {output_path}")

In [8]:
BITMAP_FILENAME = "D:\\Desktop\\csbf\\bitmap.csv"
BITMAP_IMAGE = "D:\\Desktop\\csbf\\bitmap.svg"

bucket_indices, values = read_csv(BITMAP_FILENAME)
grid = reshape_to_square(values)
create_svg_square(grid, BITMAP_IMAGE)

SVG saved to D:\Desktop\csbf\bitmap.svg
